In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer

In [ ]:
# !pip install tqdm

# from tqdm import trange, notebook     

In [ ]:
# pd.options.display.max_rows = 999999
# pd.options.display.max_columns = 500

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [2]:
# train_t = pd.read_csv('train_transaction.csv')
# train_i = pd.read_csv('train_identity.csv')
# test_t = pd.read_csv('test_transaction.csv')
# test_i = pd.read_csv('test_identity.csv')

train = pd.read_csv('train_02.csv')
test = pd.read_csv('test_02.csv')

submission = pd.read_csv('sample_submission.csv')

In [ ]:
'''
def reduce_mem_usage1(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

%time
# From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# WARNING! THIS CAN DAMAGE THE DATA 
def reduce_mem_usage2(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
'''

In [ ]:
'''
train = reduce_mem_usage2(train)
test = reduce_mem_usage2(test)
'''

In [3]:
# train = pd.merge(train_t, train_i, on='TransactionID', how='outer')
# train
train.head()

,Unnamed: 0,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,...,V304,V306,V307,V308,V310,V312,V313,V314,V315,V317
0,0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,...,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0
1,1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,...,0.0,50.0,1758.0,925.0,354.0,135.0,0.0,0.0,0.0,1404.0
4,4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# test = pd.merge(test_t, test_i, on='TransactionID', how='outer')
# test

test.head()

In [ ]:
# Label encoding할 column 추출

train.iloc[:, :100].info()
train.iloc[:, 100:200].info()
train.iloc[:, 200:300].info()
train.iloc[:, 300:400].info()
train.iloc[:, 300:400].info()

   ## float 타입 컬럼 Nan 값, KNNImputer 로 채워보기

In [4]:
train_float = train.select_dtypes('float64')
train_float

,TransactionAmt,card2,card3,card5,addr1,addr2,dist1,C1,C2,C4,...,V304,V306,V307,V308,V310,V312,V313,V314,V315,V317
0,68.50,NaN,150.0,142.0,315.0,87.0,19.0,1.0,1.0,0.0,...,0.0,0.000000,117.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,117.0
1,29.00,404.0,150.0,102.0,325.0,87.0,NaN,1.0,1.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
2,59.00,490.0,150.0,166.0,330.0,87.0,287.0,1.0,1.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,50.00,567.0,150.0,117.0,476.0,87.0,NaN,2.0,5.0,0.0,...,0.0,50.000000,1758.000000,925.000000,354.000000,135.0,0.000000,0.000000,0.000000,1404.0
4,50.00,514.0,150.0,102.0,420.0,87.0,NaN,1.0,1.0,0.0,...,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,49.00,NaN,150.0,226.0,272.0,87.0,48.0,2.0,1.0,0.0,...,0.0,0.000000,47.950001,0.000000,47.950001,0.0,47.950001,47.950001,47.950001,0.0
590536,39.50,225.0,150.0,224.0,204.0,87.0,NaN,1.0,1.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
590537,30.95,595.0,150.0,224.0,231.0,87.0,NaN,1.0,1.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
590538,117.00,481.0,150.0,224.0,387.0,87.0,3.0,1.0,1.0,0.0,...,0.0,117.000000,2903.500000,117.000000,669.500000,117.0,317.500000,669.500000,317.500000,2234.0


In [ ]:
imputer = KNNImputer(n_neighbors=3)
imputed = imputer.fit_transform(train_float)        # (np.reshape(train['card6'],(-1,1))


In [ ]:
train_float_imputed = pd.DataFrame(imputed, columns = train_float.columns)
train_float_imputed

In [ ]:
train.corr()

In [ ]:
# nan = np.nan
# X = [[1, 2, nan], [3, 4, 3], [nan, 6, 5], [8, 8, 7]]
# imputer = KNNImputer(n_neighbors=4, weights="uniform")
# imputer.fit_transform(X)

In [ ]:
print(train.isnull().sum()[:50])
print(train.isnull().sum()[50:100])
print(train.isnull().sum()[100:])

## Object 컬럼 Nan값 채우기

In [ ]:
train.select_dtypes("object").columns

In [ ]:
train_object = train.select_dtypes("object")
train_object

In [ ]:
# le = LabelEncoder()
# train['card6'] = le.fit_transform(train['card6'])
# train.card6

In [ ]:
train_object.columns

In [ ]:
imputer = KNNImputer(n_neighbors=3)
imputed = imputer.fit_transform(train_object)        # (np.reshape(train['card6'],(-1,1))
train_object_imputed = pd.DataFrame(imputed, columns = train_object.columns)
train_object_imputed

In [ ]:
train.card4

In [ ]:


X = np.array(train.card4)
imputer = KNNImputer(n_neighbors=4, weights="uniform")
imputer.fit_transform(X)

In [ ]:
train_object.card4.value_counts()

In [ ]:
(train_object['card4'] == 'visa').sum()

In [ ]:
def train_percent(a, b):
    return a / b

In [ ]:
train_percent(train_object.card4.visa / train_object.card4.count())

In [ ]:
train_object.card4.value_counts()
sns.histplot(data=train_object.card4)

In [ ]:
train_object.DeviceInfo.unique()

In [ ]:
train.card4 = le.transform

In [ ]:
train.ProductCD.unique()

In [ ]:
le.classes_

In [ ]:
le.transform(['C', 'H', 'R', 'S', 'W'])

In [ ]:
train['ProductCD'] = le.transform([train.ProductCD])
train['ProductCD']

In [ ]:
submission.shape

In [ ]:
test.ProductCD.value_counts()

In [ ]:
print(test.columns[ : 100])
print(test.columns[100 : 200])
print(test.columns[200 : 300])
print(test.columns[300 : 400])
print(test.columns[400 : 450])

In [ ]:
# 결측치 보기

test.iloc[:, :100].info()
test.iloc[:, 100:200].info()
test.iloc[:, 200:300].info()
test.iloc[:, 300:400].info()
test.iloc[:, 300:400].info()

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=train, x='ProductCD', palette='Set2')
train[['ProductCD']].value_counts() #.sort_index()
# train[['TransactionAmt']].describe()

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=test, x='card5', palette='Paired')
test[['card5']].value_counts().sort_index()
# test[['ProductCD']].describe()

In [ ]:
train.ProductCD[.value_counts()

In [ ]:
train

In [ ]:
(train['isFraud']==1).value_counts()

In [ ]:
test.dist2.value_counts().sort_index()

In [ ]:
train_C = train.filter(like='C')
train_C

In [ ]:
plt.figure(figsize=(5,5))
sns.catplot(data=train_C)

In [ ]:
scatscatterr

In [ ]:
train_i['DeviceInfo'].str.contains('SAMSUNG').sum()
# LL = train_i[train_i['DeviceInfo'].str.contains('SAMSUNG', na=True)]
# LL

In [ ]:
train.filter(like='V')

In [ ]:
# train.V1.value_counts()
plt.figure(figsize=(10,10))
sns.histplot(data=train_V)
plt.show()

In [ ]:
train_i['id']

In [ ]:
train[['P_emaildomain']]
# sns.histplot(data=train_t, x='P_emaildomain')

In [ ]:
a = set(train_t['P_emaildomain'])
b = set(train_t['R_emaildomain'])

In [ ]:
b-a

In [ ]:
print(set1.difference(set2))

In [ ]:
plt.scatter

In [ ]:
train_t.filter(regex='card2')

In [ ]:
train_t = train_t.filter(regex='addr')
train_t[:50]
# filter_.iloc[100000:100500, :]

In [ ]:
train_t.card1.unique()

In [ ]:
train_i.columns

In [ ]:
test_t[ : 50]
# print(test_t.columns[100 : 200])
# print(test_t.columns[200 : 300])
# print(test_t.columns[300 : 400])

In [ ]:
test_i.columns

In [ ]:
sns.boxplot(data=train_t)

In [ ]:
sns.catplot(data=train_t)

In [ ]:
plt.figure=figsize(10,10)
filter_.drop(columns=['ProductCD']).plot()

In [ ]:
train_t

In [ ]:
train_t[:50].info()

In [ ]:
# 구매자 도메인
train_t.P_emaildomain.value_counts().count()

In [ ]:
# 수령자 도메인
train_t.R_emaildomain.value_counts().count()

In [ ]:
train_i

In [ ]:
# train_merge = pd.merge(train_t, train_i, on='TransactionID', how='outer')
# train_merge

In [ ]:
train_merge.DeviceInfo.value_counts()

In [ ]:
test_i

In [ ]:
test_t

In [ ]:
submission

In [ ]:
train_t[train_t['isFraud'] == 1]

In [ ]:
# train_t.info()